# US Census — Florida County Population Ingestion

This notebook demonstrates how to ingest data from the U.S. Census Data API using an API key, normalize the JSON response, and persist the results as a tabular dataset.

**Dataset:** 2020 Population Estimates Program (PEP) — total population by county  
**Geography:** All counties in Florida (state FIPS = 12)  
**API:** `https://api.census.gov/data/2020/pep/population`

The workflow:

1. Configure the Census API endpoint, query parameters, and API key.
2. Call the API and retrieve county-level population data for Florida.
3. Convert the JSON response into a pandas DataFrame and clean column names.
4. Save the result as a local CSV file (`us_census_fl_county_population.csv`) for use in the GTB modernization portfolio.
5. (Later) Lift this pattern into Fabric and write the data into a Lakehouse Delta table.

In [9]:
# step_1.py — Call US Census API for Florida county population (ACS5)

import requests

CENSUS_API_KEY = "d6e29150e8ff61ccb87a6e3248f1bfff78970db5"

BASE_URL = "https://api.census.gov/data/2022/acs/acs5"

url = (
    f"{BASE_URL}"
    f"?get=NAME,B01003_001E"
    f"&for=county:*"
    f"&in=state:12"
    f"&key={CENSUS_API_KEY}"
)

response = requests.get(url)
response.raise_for_status()

print("Status:", response.status_code)
print("Preview:", response.text[:200])

data = response.json()
data[:5]

Status: 200
Preview: [["NAME","B01003_001E","state","county"],
["Alachua County, Florida","279729","12","001"],
["Baker County, Florida","27969","12","003"],
["Bay County, Florida","181055","12","005"],
["Bradford County,


[['NAME', 'B01003_001E', 'state', 'county'],
 ['Alachua County, Florida', '279729', '12', '001'],
 ['Baker County, Florida', '27969', '12', '003'],
 ['Bay County, Florida', '181055', '12', '005'],
 ['Bradford County, Florida', '27816', '12', '007']]

In [10]:
# step_2.py — Convert Census JSON response to pandas DataFrame

import pandas as pd

header = data[0]
rows = data[1:]

df = pd.DataFrame(rows, columns=header)

df = df.rename(columns={
    "NAME": "county_name",
    "B01003_001E": "population",
    "state": "state_fips",
    "county": "county_fips"
})

df["population"] = df["population"].astype(int)

df.head()

,county_name,population,state_fips,county_fips
0,"Alachua County, Florida",279729,12,001
1,"Baker County, Florida",27969,12,003
2,"Bay County, Florida",181055,12,005
3,"Bradford County, Florida",27816,12,007
4,"Brevard County, Florida",610723,12,009


In [11]:
# step_3.py — Save Census data to local CSV

output_path = "us_census_fl_county_population.csv"
df.to_csv(output_path, index=False)

output_path

'us_census_fl_county_population.csv'